In [2]:
# Install required packages
!pip install streamlit pyngrok pandas scikit-learn xgboost tensorflow --quiet

# Clean up processes
!pkill ngrok 2>/dev/null || true
!rm -rf /root/.ngrok2

app_code = """
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

def load_data():
    df = pd.read_csv("/content/drive/MyDrive/stroke_risk_dataset_v2.csv")  # Replace with your dataset path
    return df

def preprocess_data(df):
    label_encoders = {}
    for col in df.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    X = df.drop(columns=["stroke_risk_percentage", "at_risk"])
    y = df["at_risk"]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return train_test_split(X_scaled, y, test_size=0.2, random_state=42), scaler, label_encoders

def train_models(X_train, X_test, y_train, y_test):
    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(),
        "XGBoost": XGBClassifier(enable_categorical=True),
        "SVM": SVC(probability=True),
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        results[name] = (model, acc)

    nn_model = Sequential([
        Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    nn_model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)
    acc = nn_model.evaluate(X_test, y_test, verbose=0)[1]
    results["Neural Network"] = (nn_model, acc)

    best_model_name = max(results, key=lambda k: results[k][1])
    best_model = results[best_model_name][0]

    return best_model, best_model_name

def save_model(model, scaler):
    joblib.dump(model, "best_model.pkl")
    joblib.dump(scaler, "scaler.pkl")

def load_model():
    model = joblib.load("best_model.pkl")
    scaler = joblib.load("scaler.pkl")
    return model, scaler

st.title("Stroke Risk Prediction App By Umar")

df = load_data()
(X_train, X_test, y_train, y_test), scaler, label_encoders = preprocess_data(df)
best_model, best_model_name = train_models(X_train, X_test, y_train, y_test)
save_model(best_model, scaler)
st.write(f"✅ Best model selected: **{best_model_name}**")

# Instructions
st.write(\"""
### ℹ️ How to Enter Information:
- **Gender**: Enter `1` for **Male**, `0` for **Female**
- **Age**: Use the **slider** below (0–150)
- **All Other Fields**: Enter `1` for **Yes**, `0` for **No**
\""")

# Sidebar inputs
st.sidebar.header("Enter Patient Details")
input_data = []
columns = df.drop(columns=["stroke_risk_percentage", "at_risk"]).columns

for col in columns:
    if col.lower() == "age":
        value = st.sidebar.slider("Age", min_value=0, max_value=150, value=30)
    else:
        value = st.sidebar.number_input(col, min_value=0, max_value=1, value=0)
    input_data.append(value)

# Prediction
if st.sidebar.button("Predict Stroke Risk"):
    model, scaler = load_model()
    input_array = np.array(input_data).reshape(1, -1)
    input_scaled = scaler.transform(input_array)
    prediction = model.predict(input_scaled)
    result = "🔴 High Risk" if prediction[0] == 1 else "🟢 Low Risk"
    st.write(f"### 🎯 Predicted Stroke Risk: {result}")
"""

# Save the modified app code to file
with open('app.py', 'w') as f:
    f.write(app_code)

# Run the app
from pyngrok import ngrok
import subprocess

ngrok.set_auth_token("2vIWUZO3KsrfEqqficQZqwHwxxo_3dh4bmQjD7TdeA9BExntU")
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])
public_url = ngrok.connect(8501)
print("App running at:", public_url)


App running at: NgrokTunnel: "https://5a0a-35-185-162-173.ngrok-free.app" -> "http://localhost:8501"
